In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from glob import glob
import os

from importlib import reload
import utils
from scipy import stats

In [68]:
level = 'sub'
ndims = 3
fs = 12
dpi = 96

Cmap='plasma'
s = 30
sample = 'D2'
cnames = ['PC 1', 'PC 2']

wsrc = '..' + os.sep + 'cell_dams' + os.sep
nsrc = '..' + os.sep + 'nuclear_mask' + os.sep
ksrc = '..' + os.sep + 'kde' + os.sep + sample + os.sep
psrc = '..' + os.sep + 'geometry' + os.sep + sample + os.sep
isrc = '..' + os.sep + 'infected_focus_summer24' + os.sep + sample + os.sep
gsrc = '..' + os.sep + level + 'level' + os.sep + sample + os.sep

corrfiles = sorted( glob(isrc + '*'+level+'level/PI_scale*_corr_summary.csv') )
corrs = [None for _ in range(len(corrfiles))]
for i in range(len(corrs)):
    #print(corrfiles[i])
    corrs[i] = pd.read_csv(corrfiles[i], index_col='Comparison')

In [15]:
print(corrs[0].shape)
print(corrs[0].index)
print(corrs[0].columns)

(36, 10)
Index(['PC 1-vs-PC 2', 'PC 1-vs-N', 'PC 1-vs-Density', 'PC 1-vs-N(05G203100)',
       'PC 1-vs-X', 'PC 1-vs-Y', 'PC 1-vs-Cell Size', 'PC 1-vs-Eccentricity',
       'PC 2-vs-N', 'PC 2-vs-Density', 'PC 2-vs-N(05G203100)', 'PC 2-vs-X',
       'PC 2-vs-Y', 'PC 2-vs-Cell Size', 'PC 2-vs-Eccentricity',
       'N-vs-Density', 'N-vs-N(05G203100)', 'N-vs-X', 'N-vs-Y',
       'N-vs-Cell Size', 'N-vs-Eccentricity', 'Density-vs-N(05G203100)',
       'Density-vs-X', 'Density-vs-Y', 'Density-vs-Cell Size',
       'Density-vs-Eccentricity', 'N(05G203100)-vs-X', 'N(05G203100)-vs-Y',
       'N(05G203100)-vs-Cell Size', 'N(05G203100)-vs-Eccentricity', 'X-vs-Y',
       'X-vs-Cell Size', 'X-vs-Eccentricity', 'Y-vs-Cell Size',
       'Y-vs-Eccentricity', 'Cell Size-vs-Eccentricity'],
      dtype='object', name='Comparison')
Index(['slope', 'intercept', 'rvalue', 'pvalue', 'stderr', 'intercept_stderr',
       'spearson', 'ppearson', 'sspearman', 'pspearman'],
      dtype='object')


In [25]:
c = ['spearson','ppearson','sspearman','pspearman']
meta = dict()
for cname in cnames:
    for comp in ['Eccentricity', 'N(05G203100)']:
        key = cname + '-vs-' + comp
        meta[key] = pd.DataFrame(index=range(len(corrfiles)), columns=['gene', 'bw', 'scale'] + c + ['apearson', 'aspearman'])
        for i in range(len(corrs)):
            p,b = os.path.split(corrfiles[i])
            gene, bw, _ = p.split(os.sep)[-1].split('_')
            bw = int(bw[2:])
            scale = int(b.split('scale')[1].split('_')[0])
            meta[key].loc[i, ['gene','bw','scale'] ] = gene, bw, scale
            meta[key].loc[i, c] = corrs[i].loc[key, c ].values
        meta[key]['apearson'] = meta[key]['spearson'].abs()
        meta[key]['aspearman'] = meta[key]['sspearman'].abs()

In [26]:
key = 'PC 1-vs-Eccentricity'
print(key)
summary = pd.DataFrame(index = meta[key]['gene'].unique(), columns = ['bw','scale'] + c)
for gene in summary.index:
    foo = meta[key][ (meta[key]['gene'] == gene) & (meta[key]['bw'] > 10)].sort_values(by='aspearman', ascending=False).iloc[0]
    summary.loc[gene, ['bw','scale'] + c] = foo[['bw','scale'] + c]
summary

PC 1-vs-Eccentricity


,bw,scale,spearson,ppearson,sspearman,pspearman
01G164600,15,24,-0.120472,0.059713,-0.121775,0.056985
05G092200,15,48,0.394581,0.0,0.38075,0.0
05G203100,15,40,-0.13127,0.005766,-0.163983,0.000546
05G216000,20,40,-0.197333,0.0,-0.208798,0.0
12G027400,15,48,-0.103649,0.001673,-0.120962,0.000241
15G210100,30,48,-0.066203,0.045874,-0.085436,0.009924
17G185800,15,8,0.171813,0.0,0.162543,0.000001
17G195900,25,16,0.531839,0.0,0.573112,0.0
BAC45727,30,48,-0.188808,0.0,-0.214549,0.0
BAC47034,20,16,-0.256658,0.0,-0.266103,0.0


In [8]:
key = 'PC 1-vs-N(05G203100)'
print(key)
summary = pd.DataFrame(index = meta[key]['gene'].unique(), columns = ['bw','scale'] + c)
for gene in summary.index:
    foo = meta[key][ (meta[key]['gene'] == gene) & (meta[key]['bw'] > 10)].sort_values(by='aspearman', ascending=False).iloc[0]
    summary.loc[gene, ['bw','scale'] + c] = foo[['bw','scale'] + c]
summary

PC 1-vs-N(05G203100)


,bw,scale,spearson,ppearson,sspearman,pspearman
01G164600,30,48,-0.005582,0.930733,0.154077,0.015788
05G092200,15,8,0.151708,0.000004,0.145364,0.00001
05G203100,15,8,0.904319,0.0,0.895361,0.0
05G216000,15,8,0.198476,0.0,0.1997,0.0
12G027400,25,8,0.248808,0.0,0.317669,0.0
15G210100,20,48,0.102117,0.00204,0.182029,0.0
17G185800,20,48,0.066186,0.046412,0.145026,0.000012
17G195900,15,16,0.050186,0.128651,0.165851,0.0
BAC45727,15,24,0.150461,0.000005,0.216993,0.0
BAC47034,15,8,0.136833,0.000032,0.171169,0.0


In [29]:
key = 'PC 2-vs-N(05G203100)'
print(key)
summary = pd.DataFrame(index = meta[key]['gene'].unique(), columns = ['bw','scale'] + c)
for gene in summary.index:
    foo = meta[key][ (meta[key]['gene'] == gene) & (meta[key]['bw'] > 10)].sort_values(by='aspearman', ascending=False).iloc[0]
    summary.loc[gene, ['bw','scale'] + c] = foo[['bw','scale'] + c]
summary

PC 2-vs-N(05G203100)


,bw,scale,spearson,ppearson,sspearman,pspearman
01G164600,15,16,0.038028,0.553585,0.11191,0.080429
05G092200,15,8,0.017,0.606969,-0.078364,0.017562
05G203100,20,40,0.473746,0.0,0.526481,0.0
05G216000,15,48,0.106613,0.00136,0.125014,0.00017
12G027400,30,40,0.240961,0.0,0.261089,0.0
15G210100,20,8,-0.062357,0.060063,-0.150417,0.000005
17G185800,30,8,-0.012151,0.714921,-0.100499,0.002458
17G195900,20,8,-0.116828,0.00039,-0.153242,0.000003
BAC45727,15,8,0.008037,0.808076,-0.110472,0.000811
BAC47034,15,16,0.099218,0.002617,0.161471,0.000001


In [30]:
key = 'PC 2-vs-Eccentricity'
print(key)
summary = pd.DataFrame(index = meta[key]['gene'].unique(), columns = ['bw','scale'] + c)
for gene in summary.index:
    foo = meta[key][ (meta[key]['gene'] == gene) & (meta[key]['bw'] > 10)].sort_values(by='aspearman', ascending=False).iloc[0]
    summary.loc[gene, ['bw','scale'] + c] = foo[['bw','scale'] + c]
summary

PC 2-vs-Eccentricity


,bw,scale,spearson,ppearson,sspearman,pspearman
01G164600,15,8,-0.116498,0.068705,-0.203972,0.001327
05G092200,25,8,0.332353,0.0,0.331335,0.0
05G203100,25,32,0.056742,0.23438,0.125894,0.008125
05G216000,15,8,-0.208621,0.0,-0.170509,0.0
12G027400,20,40,0.153235,0.000003,0.168017,0.0
15G210100,15,48,0.23687,0.0,0.239256,0.0
17G185800,15,48,0.153011,0.000004,0.179343,0.0
17G195900,15,24,0.576372,0.0,0.606246,0.0
BAC45727,15,8,-0.187766,0.0,-0.195629,0.0
BAC47034,20,8,0.372306,0.0,0.3673,0.0


# Other 

In [75]:
cidx = 169
geometry = pd.read_csv(psrc + genename + os.sep + '26_bins_peripherality_c{:06d}.csv'.format(cidx))
geometry[['{}_{}'.format(x,bw) for x in ['mean','sum']]]

,mean_15,sum_15
0,4.216592e-08,0.000724
1,6.207975e-08,0.001210
2,8.365267e-08,0.001518
3,1.129211e-07,0.002042
4,1.327085e-07,0.002242
5,1.577029e-07,0.002439
6,1.708979e-07,0.002700
7,1.807158e-07,0.002660
8,1.914771e-07,0.002578
9,1.824757e-07,0.002369


In [65]:
smcolumns=['PC 1', 'PC 2', 'N', 'Density', 'N(05G203100)', 'X', 'Y', 'Cell Size', 'Eccentricity']
cnames = ['N', 'Density', 'PC 1', 'PC 2']
k = 0
vs = ['' for _ in range( len(smcolumns)*( len(smcolumns) - 1)//2) ]
for i in range(len(smcolumns)-1):
    for j in range(i+1, len(smcolumns)):
        vs[k] = '{}-vs-{}'.format(smcolumns[i] , smcolumns[j] )
        k += 1
print(vs)

['PC 1-vs-PC 2', 'PC 1-vs-N', 'PC 1-vs-Density', 'PC 1-vs-N(05G203100)', 'PC 1-vs-X', 'PC 1-vs-Y', 'PC 1-vs-Cell Size', 'PC 1-vs-Eccentricity', 'PC 2-vs-N', 'PC 2-vs-Density', 'PC 2-vs-N(05G203100)', 'PC 2-vs-X', 'PC 2-vs-Y', 'PC 2-vs-Cell Size', 'PC 2-vs-Eccentricity', 'N-vs-Density', 'N-vs-N(05G203100)', 'N-vs-X', 'N-vs-Y', 'N-vs-Cell Size', 'N-vs-Eccentricity', 'Density-vs-N(05G203100)', 'Density-vs-X', 'Density-vs-Y', 'Density-vs-Cell Size', 'Density-vs-Eccentricity', 'N(05G203100)-vs-X', 'N(05G203100)-vs-Y', 'N(05G203100)-vs-Cell Size', 'N(05G203100)-vs-Eccentricity', 'X-vs-Y', 'X-vs-Cell Size', 'X-vs-Eccentricity', 'Y-vs-Cell Size', 'Y-vs-Eccentricity', 'Cell Size-vs-Eccentricity']


In [63]:
metacell = pd.read_csv(ksrc + sample + '_cells_metadata.csv', index_col='ndimage_cellID')
metaecc = pd.read_csv(ksrc + sample + '_nodule_root_eccentricity.csv', index_col='ndimage_cellID')
cell_nuc = pd.read_csv(ksrc + sample + '_nuclei_limits.csv')

In [48]:
stepsize = 3
exclude_nuclei = True

if exclude_nuclei:
    filenameb = '..' + os.sep + 'translocs' + os.sep + sample + os.sep + 'location_corrected_D2_-_{}.csv'
    ex_nuclei = ''
    pdkw = {'header':None, 'names':['X', 'Y', 'Z']}
else:
    filenameb = '..' + os.sep + 'Bacteria Info for Erik' + os.sep + '{}_v2.txt'
    ex_nuclei = '_w_nucleus'
    pdkw = {'sep':'\t'}

transcell = pd.read_csv(ksrc + sample + '_transcells_metadata' + ex_nuclei + '.csv', index_col='gene').rename(columns=int)
transcriptomes = np.asarray(transcell.index, dtype=str)
Genes = utils.get_range_gene_values(isrc + 'genes_to_focus_infection' + ex_nuclei + '.csv', transcriptomes, startval=0)

translocs = dict()
for key in transcriptomes:
    filename = filenameb.format(key)
    translocs[key] = pd.read_csv(filename, **pdkw)
    
tlocs = pd.concat(translocs)
zmax = np.max(tlocs['Z']+stepsize)
zbins = np.arange(0, zmax+stepsize, stepsize)

In [57]:
if 'BAC' in gene:
    geneprint = 'Bac.' + gene[3:]
    genename = gene
else:
    geneprint = 'Glyma.' + gene
    genename = 'GLYMA_' + gene
bw, SCALE = summary.loc[gene, ['bw', 'scale']].values.astype(int)
bname = isrc + '{}_bw{}_{}level'.format(gene, bw, level) + os.sep + 'PI_scale{}_'.format(SCALE)
Bname = geneprint + ' PIs: KDE bandwidth {}. {}level persistence. Scale {}'.format(bw, level.title(), SCALE)
print(bname, Bname, sep='\n')

../infected_focus_summer24/D2/BAC52602_bw15_sublevel/PI_scale8_
Bac.52602 PIs: KDE bandwidth 15. Sublevel persistence. Scale 8


In [54]:
hdims = np.array([1,2])
Pname = ' [$' + ' \\oplus '.join(['H_{}'.format(k) for k in hdims]) + '$]'
pname = 'H' + '+'.join(hdims.astype(str))
embedding = pd.read_csv(bname + 'pca.csv')
zero_val = embedding.iloc[-1, 2:].values
zs = [0, 0, zero_val[0], zero_val[1]]
embedding = embedding.iloc[:-1]
pca = embedding.iloc[:,2:4].values

In [66]:
cmask = embedding['ndimage_ID'].copy()

summary0 = embedding.iloc[:, 1:4].copy().set_index('ndimage_ID')
density = transcell.loc[genename, cmask] / metacell.loc[cmask, 'cyto_area']
density.name = 'density'
NN = transcell.loc[genename, cmask].copy()
NN.name = 'N'
summary0 = summary0.join(pd.concat([NN, density], axis=1), how='outer').join(transcell.loc['GLYMA_05G203100', cmask] + 1)
summary0 = summary0.join(metacell.loc[cmask, ['ndimage_comX','ndimage_comY','cyto_area']], how='outer')
summary0.iloc[ pd.isna(summary0.iloc[:,0]).values , :2] = zero_val[:2]
summary0 = summary0.join(metaecc['eccentricity'].max() - metaecc.loc[cmask, 'eccentricity'])

summary0.columns = smcolumns
print(summary0.shape)
summary0.head()

(918, 9)


,PC 1,PC 2,N,Density,N(05G203100),X,Y,Cell Size,Eccentricity
ndimage_ID,,,,,,,,,
169,-0.260521,-0.247730,94,0.000880,4,7789.38,3043.66,106788,4745.936000
188,1.334498,-0.732867,167,0.000705,22,8140.11,3270.17,236885,4455.824132
199,0.999605,0.183603,203,0.001983,1,7409.04,3189.21,102388,4677.492017
209,-1.224072,-0.072569,29,0.000680,1,8798.15,3142.66,42671,4555.942172
224,-0.031961,-0.214672,96,0.001215,1,6297.28,3275.46,79031,4832.496197


In [67]:
x = summary0['PC 2']
y = summary0['Eccentricity']
stats.spearmanr(x,y)

SignificanceResult(statistic=-0.219029569167738, pvalue=1.9615828765844545e-11)

In [101]:
transcell[Cells].iloc[:10].T.describe()

gene,BAC45727,BAC46169,BAC47034,BAC47856,BAC48395,BAC51072,BAC51722,BAC52602,BAC52793,BAC52805
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,31.531590,1.063181,1788.593682,1.887800,2.883442,2.704793,0.311547,117.250545,2.364924,1.897603
std,20.124221,1.191727,1033.164432,2.009097,2.441134,2.427352,0.615058,63.086328,2.271331,1.746277
min,0.000000,0.000000,77.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000
25%,16.000000,0.000000,964.250000,0.000000,1.000000,1.000000,0.000000,71.000000,1.000000,1.000000
50%,27.000000,1.000000,1655.000000,1.000000,2.000000,2.000000,0.000000,108.000000,2.000000,1.000000
75%,44.000000,2.000000,2466.250000,3.000000,4.000000,4.000000,0.000000,157.000000,3.000000,3.000000
max,113.000000,7.000000,5641.000000,17.000000,14.000000,14.000000,3.000000,409.000000,20.000000,9.000000
